In [1]:
import numpy as np
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model

from Source import RM

import tensorflow as tf

Using TensorFlow backend.


In [2]:
m = 4
r = 1
length = 2**m

In [3]:
label = np.eye(32)
msg=np.zeros((32,5))
for i in range(32):
    a = list(format(i, 'b').zfill(5))
    msg[i] = a

In [4]:
G = RM.G_matrix(length, m, r)

In [5]:
model = load_model('Model/model_(1,4)_p6_2bits_softmax.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
def add_masking(received_code, msg_size):
    masking = np.ones((msg_size,2))
    if np.var(received_code) > 1.9:
        received_code = np.hstack((masking, received_code))
    elif np.var(received_code) > 1.45:
        masking[:,1] = -1*masking[:,1]
        received_code = np.hstack((masking, received_code))
    elif np.var(received_code) > 1.225:
        masking[:,0] = -1*masking[:,0]
        received_code = np.hstack((masking, received_code))
    else:
        masking = -1 * masking
        received_code = np.hstack((masking, received_code))
    return received_code

In [7]:
for i in range(10**4):
    send_code, received_code = RM.Encoding(msg, G, 0, length)
    test_send_code, test_received_code = RM.Encoding(msg, G, 0, length)
            
    received_code = add_masking(received_code, msg.shape[0])
    test_received_code = add_masking(test_received_code, msg.shape[0])
    
    label = np.eye(32)
    
    for j in range(1,11):
        send_code, add_received_code = RM.Encoding(msg, G, j, length)
        test_send_code, add_test_received_code = RM.Encoding(msg, G, j, length)
        
        add_received_code = add_masking(add_received_code, msg.shape[0])
        add_test_received_code = add_masking(add_test_received_code, msg.shape[0])
        
        received_code = np.vstack((received_code, add_received_code))
        test_received_code = np.vstack((test_received_code, add_test_received_code))
        
        add_label = np.eye(32)
        label = np.vstack((label,add_label))
    
    
    send_code, add_received_code = RM.Encoding(msg, G, j, length)
    test_send_code, add_test_received_code = RM.Encoding(msg, G, j, length)
    
    add_received_code = add_masking(send_code, msg.shape[0])
    add_test_received_code = add_masking(test_send_code, msg.shape[0])
    
    received_code = np.vstack((received_code, add_received_code))
    test_received_code = np.vstack((test_received_code, add_test_received_code))
    
    label = np.vstack((label, add_label))
    
    model.fit(received_code, label,
              epochs=10,
              batch_size=10000,
              shuffle=True,
              validation_data=(test_received_code, label), verbose = 0)
        
    if i%10**3==0: 
        label = np.eye(32)
        test_send_code, test_received_code = RM.Encoding(msg, G, i%10, length)
        
        test_received_code = add_masking(test_received_code, msg.shape[0])
                
        loss = model.evaluate(test_received_code, label)
        print(loss)
        
        model.save('Model/model_(1,4)_p6_2bits_softmax.h5')

32/32 [==============================] - 0s 31us/step
[0.07021009176969528, 0.9814453125]
32/32 [==============================] - 0s 125us/step
[0.023540694266557693, 0.9892578125]
32/32 [==============================] - 0s 94us/step
[0.025355752557516098, 0.9921875]
32/32 [==============================] - 0s 94us/step
[0.02933204546570778, 0.9931640625]
32/32 [==============================] - 0s 93us/step
[0.03139418363571167, 0.990234375]
32/32 [==============================] - 0s 94us/step
[0.08277789503335953, 0.9755859375]
32/32 [==============================] - 0s 93us/step
[0.06391678005456924, 0.984375]
32/32 [==============================] - 0s 125us/step
[0.04824203997850418, 0.9833984375]
32/32 [==============================] - 0s 93us/step
[0.032772041857242584, 0.9912109375]
32/32 [==============================] - 0s 62us/step
[0.017052339389920235, 0.99609375]


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-30eaafcf7813>", line 39, in <module>
    validation_data=(test_received_code, label), verbose = 0)
  File "D:\anaconda\lib\site-packages\keras\engine\training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "D:\anaconda\lib\site-packages\keras\engine\training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "D:\anaconda\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3476, in __call__
    run_metadata=self.run_metadata)
  File "D:\anaconda\lib\site-packages\tensorflow_core\python\client\session.py", line 1472, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\IPython\core\int

KeyboardInterrupt: 

In [ ]:
model.save('Model/model_(1,4)_p6_2bits_softmax.h5')

In [ ]:
loss = model.predict(received_code)
print(np.reshape(loss.argmax(axis=1),(-1,32)))